In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt

c = 9999
g = 1/15
b = 0.01
l = 0.25
g0 = 1/15
c0 = 1/16
ls = 0.25

def bracket(t):
    value = 1 + c*math.exp((g-l)*t)
    value = value/(1 + c)
    return value

def mintegrand(t):
    power = l/(l-g)
    phi = bracket(t)
    value = ((1 + c)*b/g)*(phi - math.pow(phi,power)*math.exp(g*t)) + 1 + c*math.exp((g-l)*t)
    integrand = 1/value
    return integrand

def pintegrand(s):
    value = ls/(1 + c*mvalue(1,s))
    return value

def jintegrand(s):
    value = ls*c*mvalue(-1,s)/(1 + c*mvalue(-1,s)) - (g0 - c0*mintegrand(s))
    return value

def mvalue(parity,s):
    i = quad(mintegrand,0,s)
    temp = c0*i[0] + (ls - g0)*s
    #print(temp)
    value = math.exp(parity*temp)
    return value

def pvalue(t):
    i = quad(pintegrand,0,t)
    value = math.exp(-1*i[0])
    return value

def jvalue(t):
    i = quad(jintegrand,0,t)
    value = math.exp(i[0])
    return value

def main():
    I = 500
    S = 9500
    infected = [I]
    susceptible = [S]
    time = [0]
    for t in range(50):
        infected.append(I*jvalue(t+1))
        susceptible.append(S*pvalue(t+1))
        time.append(t+1)
    plt.plot(time,infected,label='Infected')
    plt.plot(time,susceptible,label='Susceptible')
    plt.legend()
    plt.show()
    
main()
    

In [ ]:
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt

I = 1
S = 9999
c = S/I
g = 1/15
b = 1/20
l = 0.25

def infectedfast(t):
    power = l/(g-l)
    value = (1 + c*math.exp((g-l)*t))/(1 + c)
    final = math.pow(value,power)*math.exp(-g*t)
    return I*final

def susceptiblefast(t):
    power = l/(g-l)
    value = (math.exp((l-g)*t) + c)/(1 + c)
    final = math.pow(value,power)
    return S*final

def recoveredfast(t):
    i = quad(infectedfast,0,t)
    final = b*i[0]
    #power = g/(g-l)
    #value = (1 + c*math.exp((g-l)*t))/(1 + c)
    #final = ((1 + c)*b/g)*(math.exp(-g*t)*math.pow(value,power)-1)
    return final

def main1():
    infected = [I]
    susceptible = [S]
    recovered = [0]
    time = [0]
    for t in range(100):
        infected.append(infectedfast(t+1))
        susceptible.append(susceptiblefast(t+1))
        recovered.append(recoveredfast(t+1))
        time.append(t+1)
    plt.plot(time,infected,label='Infected')
    plt.plot(time,susceptible,label='Susceptible')
    plt.plot(time,recovered,label='Recovered')
    plt.legend()
    plt.show()
    return

g0 = 1/15
c0 = 1/17
ls = 0.25

def mintegrand(t):
    value = ls - g0 + c0*(infectedfast(t))/(infectedfast(t)+susceptiblefast(t))
    return value

def pintegrand(s):
    value = ls/(1 + c*mvalue(-1,s))
    return value

def jintegrand(s):
    value = ls*c*mvalue(-1,s)/(1 + c*mvalue(-1,s)) - (g0 - c0*(infectedfast(s))/(infectedfast(s)+susceptiblefast(s)))
    return value

def mvalue(parity,s):
    i = quad(mintegrand,0,s)
    #print(temp)
    value = math.exp(parity*i[0])
    #print(i[0])
    return value

def pvalue(t):
    k = -1
    i = quad(pintegrand,0,t)
    value = math.exp(k*i[0])
    return value

def jvalue(t):
    i = quad(jintegrand,0,t)
    value = math.exp(i[0])
    return value

def main2():
    infected = [I]
    susceptible = [S]
    time = [0]
    for t in range(500):
        infected.append(I*jvalue(t+1))
        susceptible.append(S*pvalue(t+1))
        time.append(t+1)
    plt.plot(time,infected,label='Infected')
    plt.plot(time,susceptible,label='Susceptible')
    plt.legend()
    plt.show()
    return
#currently using same model for (fast) and (slow without fast)
enter = 10
def main3():
    global I
    global S
    global c
    infected = [I]
    susceptible = [S]
    time = [0]
    for t in range(500):
        if t <= enter:
            infected.append(infectedfast(t+1))
            susceptible.append(susceptiblefast(t+1))
        if t > enter:
            I = infected[enter]
            S = susceptible[enter]
            c = S/I
            infected.append(I*jvalue(t-enter))
            susceptible.append(S*pvalue(t-enter))
        time.append(t+1)
    plt.plot(time,infected,label='Infected')
    plt.plot(time,susceptible,label='Susceptible')
    plt.legend()
    plt.title(str(enter))
    plt.show()
    return

main1()
main2()
while enter < 200:
    main3()
    enter += 10
    I = 1
    S = 9999
    c = S/I

In [ ]:
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt

def infectedfast(t,g,l,c,I_f):
    power = l/(g-l)
    value = (1 + c*math.exp((g-l)*t))/(1 + c)
    final = math.pow(value,power)*math.exp(-g*t)
    return I_f*final

def susceptiblefast(t,g,l,c,S_f):
    power = l/(g-l)
    value = (math.exp((l-g)*t) + c)/(1 + c)
    final = math.pow(value,power)
    return S_f*final

def recoveredfast(t,g,l,b,c,I_f):
    i = quad(infectedfast,0,t,args=(g,l,c,I_f))
    final = b*i[0]
    #power = g/(g-l)
    #value = (1 + c*math.exp((g-l)*t))/(1 + c)
    #final = ((1 + c)*b/g)*(math.exp(-g*t)*math.pow(value,power)-1)
    return final

def mintegrand(t,ls,g0,a0,g,l,c_f,I_f):
    c0 = a0*g0
    value = ls - (max(0.0001,g0 - c0*(infectedfast(t,g,l,c_f,I_f))))
    return value

def pintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    value = ls/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f))
    return value

def jintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    c0 = a0*g0
    value = ls*c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)) - max(0.0001,(g0 - c0*(infectedfast(s,g,l,c_f,I_f))))
    return value

def mvalue(parity,s,ls,g0,a0,g,l,c_f,I_f):
    i = quad(mintegrand,0,s,args=(ls,g0,a0,g,l,c_f,I_f))
    #print(temp)
    value = math.exp(parity*i[0])
    #print(i[0])
    return value

def pvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    k = -1
    i = quad(pintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(k*i[0])
    return value

def jvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    i = quad(jintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(i[0])
    return value


def main3(eta,zeta,a0,enter):
    I_f = 1
    S_f = 9999
    c_f = S_f/I_f
    I_s = 1
    S_s = 9999
    c_s = S_s/I_s
    g = 1/15
    b = 1/20
    l = 0.25
    ls = eta*l
    g0 = zeta*g
  
    infected = [I_s]
    susceptible = [S_s]
    Ifast = [0]
    Sfast = [S_f]
    time = [0]
    for t in range(300):
        if t <= enter:
            if t < enter:
                Ifast.append(Ifast[t-1])
                Sfast.append(Sfast[t-1])
            elif t == enter:
                Ifast.append(I_f)
                Sfast.append(S_f)
            infected.append(infectedfast(t+1,g0,ls,c_s,I_s))
            susceptible.append(susceptiblefast(t+1,g0,ls,c_s,S_s))
        if t > enter:
            Ifast.append(infectedfast(t-enter,g,l,c_f,I_f))
            Sfast.append(susceptiblefast(t-enter,g,l,c_f,S_f))
            I_s = infected[enter]
            S_s = susceptible[enter]
            c_s = S_s/I_s
            infected.append(I_s*jvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
            susceptible.append(S_s*pvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
        time.append(t+1)
    plt.plot(time,infected,label='Infected')
    plt.plot(time,susceptible,label='Susceptible')
    plt.plot(time,Ifast,label='FastInfected')
    plt.plot(time,Sfast,label='FastSusceptible')
    plt.legend()
    plt.title('[+} Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0))
    plt.savefig('Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0)+'.png')
    plt.show()
    return

for j in range(6):
    for i in range(7):
        main3(0.5,0.6,0.0001 + (0.0002*j),(i*20)+40)

In [ ]:
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt

def infectedfast(t,g,l,c,I_f):
    power = l/(g-l)
    value = (1 + c*math.exp((g-l)*t))/(1 + c)
    final = math.pow(value,power)*math.exp(-g*t)
    return I_f*final

def susceptiblefast(t,g,l,c,S_f):
    power = l/(g-l)
    value = (math.exp((l-g)*t) + c)/(1 + c)
    final = math.pow(value,power)
    return S_f*final

def recoveredfast(t,g,l,b,c,I_f):
    i = quad(infectedfast,0,t,args=(g,l,c,I_f))
    final = b*i[0]
    #power = g/(g-l)
    #value = (1 + c*math.exp((g-l)*t))/(1 + c)
    #final = ((1 + c)*b/g)*(math.exp(-g*t)*math.pow(value,power)-1)
    return final

def mintegrand(t,ls,g0,a0,g,l,c_f,I_f):
    c0 = a0*g0
    value = ls - (max(0.0001,g0 - c0*(infectedfast(t,g,l,c_f,I_f))))
    return value

def pintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    value = ls/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f))
    return value

def jintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    c0 = a0*g0
    value = ls*c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)) - max(0.0001,(g0 - c0*(infectedfast(s,g,l,c_f,I_f))))
    return value

def mvalue(parity,s,ls,g0,a0,g,l,c_f,I_f):
    i = quad(mintegrand,0,s,args=(ls,g0,a0,g,l,c_f,I_f))
    #print(temp)
    value = math.exp(parity*i[0])
    #print(i[0])
    return value

def pvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    k = -1
    i = quad(pintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(k*i[0])
    return value

def jvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    i = quad(jintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(i[0])
    return value

colors = ['b','g','r','c','m','y','k','w']

def main3(eta,zeta,a0):
    I_f = 1
    S_f = 9999
    c_f = S_f/I_f
    I_s = 1
    S_s = 9999
    c_s = S_s/I_s
    g = 1/15
    b = 1/20
    l = 0.25
    ls = eta*l
    g0 = zeta*g
  
    
   
    for i in range(6):
        enter = (i*40)+140
        if i == 5:
            enter =601
        infected = [I_s]
        susceptible = [S_s]
        Ifast = [0]
        Sfast = [S_f]
        time = [0]
        for t in range(600):
            if t <= enter:
                if t < enter:
                    Ifast.append(Ifast[t-1])
                    Sfast.append(Sfast[t-1])
                elif t == enter:
                    Ifast.append(I_f)
                    Sfast.append(S_f)
                infected.append(infectedfast(t+1,g0,ls,c_s,I_s))
                susceptible.append(susceptiblefast(t+1,g0,ls,c_s,S_s))
            if t > enter:
                Ifast.append(infectedfast(t-enter,g,l,c_f,I_f))
                Sfast.append(susceptiblefast(t-enter,g,l,c_f,S_f))
                I_s = infected[enter]
                S_s = susceptible[enter]
                c_s = S_s/I_s
                infected.append(I_s*jvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
                susceptible.append(S_s*pvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
            time.append(t+1)
        plt.plot(time,infected,label='t = '+str(enter),color = colors[i])
        #plt.plot(time,susceptible,label='Susceptible')
        #plt.plot(time,Ifast,label='FastInfected')
        #plt.plot(time,Sfast,label='FastSusceptible')
        plt.legend()
        plt.title('[+}Infected Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0))
        #plt.savefig('Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0)+'.png')
        I_s = 1
        S_s = 9999
        c_s = S_s/I_s
    plt.show()
    return

main3(0.5,1.5,0.0001)
main3(0.5,1.5,0.0007)
main3(0.5,1.5,0.001)

In [ ]:
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt

def infectedfast(t,g,l,c,I_f):
    power = l/(g-l)
    value = (1 + c*math.exp((g-l)*t))/(1 + c)
    final = math.pow(value,power)*math.exp(-g*t)
    return I_f*final

def susceptiblefast(t,g,l,c,S_f):
    power = l/(g-l)
    value = (math.exp((l-g)*t) + c)/(1 + c)
    final = math.pow(value,power)
    return S_f*final

def recoveredfast(t,g,l,b,c,I_f):
    i = quad(infectedfast,0,t,args=(g,l,c,I_f))
    final = b*i[0]
    #power = g/(g-l)
    #value = (1 + c*math.exp((g-l)*t))/(1 + c)
    #final = ((1 + c)*b/g)*(math.exp(-g*t)*math.pow(value,power)-1)
    return final

def mintegrand(t,ls,g0,a0,g,l,c_f,I_f):
    c0 = a0*g0
    value = ls - (max(0.0001,g0 - c0*(infectedfast(t,g,l,c_f,I_f))))
    return value

def pintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    value = ls/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f))
    return value

def jintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    c0 = a0*g0
    value = ls*c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)) - max(0.0001,(g0 - c0*(infectedfast(s,g,l,c_f,I_f))))
    return value

def mvalue(parity,s,ls,g0,a0,g,l,c_f,I_f):
    i = quad(mintegrand,0,s,args=(ls,g0,a0,g,l,c_f,I_f))
    #print(temp)
    value = math.exp(parity*i[0])
    #print(i[0])
    return value

def pvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    k = -1
    i = quad(pintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(k*i[0])
    return value

def jvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    i = quad(jintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(i[0])
    return value


def main3(eta,zeta,a0,enter):
    I_f = 1
    S_f = 9999
    c_f = S_f/I_f
    I_s = 1
    S_s = 9999
    c_s = S_s/I_s
    g = 1/15
    b = 1/20
    l = 0.25
    ls = eta*l
    g0 = zeta*g
  
    infected = [I_s]
    susceptible = [S_s]
    Ifast = [0]
    Sfast = [S_f]
    time = [0]
    for t in range(2000):
        if t <= enter:
            if t < enter:
                Ifast.append(Ifast[t-1])
                Sfast.append(Sfast[t-1])
            elif t == enter:
                Ifast.append(I_f)
                Sfast.append(S_f)
            infected.append(infectedfast(t+1,g0,ls,c_s,I_s))
            susceptible.append(susceptiblefast(t+1,g0,ls,c_s,S_s))
        if t > enter:
            Ifast.append(infectedfast(t-enter,g,l,c_f,I_f))
            Sfast.append(susceptiblefast(t-enter,g,l,c_f,S_f))
            I_s = infected[enter]
            S_s = susceptible[enter]
            c_s = S_s/I_s
            infected.append(I_s*jvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
            susceptible.append(S_s*pvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
        time.append(t+1)
    plt.plot(time,infected,label='Epidemic')
    #plt.plot(time,susceptible,label='Susceptible')
    plt.plot(time,Ifast,label='Pandemic')
    #plt.plot(time,Sfast,label='FastSusceptible')
    plt.legend()
    plt.title('[+} Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0))
    plt.savefig('Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0)+'.png')
    plt.show()
    return


main3(0.08,0.07,0.0001,400)

In [ ]:
import matplotlib.pyplot as plt
import math

def normalinfected(t):
    value = I_0*((1 + C_0*math.exp((g-l)*t))/(1 + C_0))**(l/(g-l))*math.exp(-g*t)
    return value
def normalsusceptible(t):
    value = S_0*((math.exp((l-g)*t) + C_0)/(1 + C_0))**(l/(g-l))
    return value
def newinfected(t):
    value = I_entry*math.exp(l*t)*(((g-l-g*a)*C_entry*t + C_entry + 1)/(1 + C_entry))**(-l/((g-l-g*a)*C_entry)) - (g*t - g*a*(t + (t**2)*(l_f - g_f)/2))
    return value
def newsusceptible(t):
    value = S_entry*(((g-l-g*a)*C_entry*t + C_entry + 1)/(1 + C_entry))**(-l/((g-l-g*a)*C_entry))
    return value

g_f = 1/15
l_f = 0.25
I_0 = 1
S_0 = 9999
C_0 = S_0/I_0

def main(entry,eta,zeta,a0):
    global l
    l = eta*l_f
    global g
    g = zeta*g_f
    global a
    a = a0
    global I_entry
    I_entry = normalinfected(entry)
    global S_entry
    S_entry = normalsusceptible(entry)
    global C_entry
    C_entry = I_entry/S_entry

    time1 = []
    normal = []
    time = []
    infected = []
    for t in range(50501):
        time.append((t + 1 - 5251)/1000)
        #time1.append(t+1)
        #normal.append(normalinfected(t+1))
        if t <5251:
            infected.append(normalinfected(entry + (t + 1 - 5251)/1000))
        else:
            infected.append(newinfected((t + 1 - 5251)/1000))
    plt.plot(time,infected)
    plt.title('[+} Time = '+str(entry)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a))
    plt.show()
    return

for i in range(20):
    main(300 + i*25,0.08,0.07,0.0001)


In [ ]:
from scipy.integrate import quad
from scipy.integrate import trapz
import math
import matplotlib.pyplot as plt

def infectedfast(t,g,l,c,I_f):
    power = l/(g-l)
    value = (1 + c*math.exp((g-l)*t))/(1 + c)
    final = math.pow(value,power)*math.exp(-g*t)
    return I_f*final

def susceptiblefast(t,g,l,c,S_f):
    power = l/(g-l)
    value = (math.exp((l-g)*t) + c)/(1 + c)
    final = math.pow(value,power)
    return S_f*final

def recoveredfast(t,g,l,b,c,I_f):
    i = quad(infectedfast,0,t,args=(g,l,c,I_f))
    final = b*i[0]
    #power = g/(g-l)
    #value = (1 + c*math.exp((g-l)*t))/(1 + c)
    #final = ((1 + c)*b/g)*(math.exp(-g*t)*math.pow(value,power)-1)
    return final

def mintegrand(t,ls,g0,a0,g,l,c_f,I_f):
    c0 = a0*g0
    value = ls - (max(0.0001,g0 - c0*(infectedfast(t,g,l,c_f,I_f))))
    return value

def pintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    value = ls/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f))
    return value

def jintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    c0 = a0*g0
    value = ls*c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)) - max(0.0001,(g0 - c0*(infectedfast(s,g,l,c_f,I_f))))
    return value

def mvalue(parity,s,ls,g0,a0,g,l,c_f,I_f):
    i = quad(mintegrand,0,s,args=(ls,g0,a0,g,l,c_f,I_f))
    #print(temp)
    value = math.exp(parity*i[0])
    #print(i[0])
    return value

def pvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    k = -1
    i = quad(pintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(k*i[0])
    return value

def jvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    i = quad(jintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(i[0])
    return value

area = []
xvalues = []

def main3(eta,zeta,a0,enter):
    I_f = 1
    S_f = 9999
    c_f = S_f/I_f
    I_s = 1
    S_s = 9999
    c_s = S_s/I_s
    g = 1/15
    b = 1/20
    l = 0.25
    ls = eta*l
    g0 = zeta*g
  
    infected = [I_s]
    susceptible = [S_s]
    Ifast = [0]
    Sfast = [S_f]
    time = [0]
    for t in range(1500):
        if t <= enter:
            if t < enter:
                Ifast.append(Ifast[t-1])
                Sfast.append(Sfast[t-1])
            elif t == enter:
                Ifast.append(I_f)
                Sfast.append(S_f)
            infected.append(infectedfast(t+1,g0,ls,c_s,I_s))
            susceptible.append(susceptiblefast(t+1,g0,ls,c_s,S_s))
        if t > enter:
            Ifast.append(infectedfast(t-enter,g,l,c_f,I_f))
            Sfast.append(susceptiblefast(t-enter,g,l,c_f,S_f))
            I_s = infected[enter]
            S_s = susceptible[enter]
            c_s = S_s/I_s
            infected.append(I_s*jvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
            susceptible.append(S_s*pvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
        time.append(t+1)
    plt.plot(time,infected,label='Infected')
    area.append(trapz(infected,time))
    xvalues.append(enter)
    #plt.plot(time,susceptible,label='Susceptible')
    plt.plot(time,Ifast,label='FastInfected')
    #plt.plot(time,Sfast,label='FastSusceptible')
    plt.legend()
    plt.title('[+} Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0))
    #plt.savefig('Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0)+'.png')
    plt.show()
    return


def run():
    for i in range(8):
        main3(0.08,0.07,0.0007,200+i*100)

    plt.scatter(xvalues,area)
    plt.ylabel('Total no. of infections')
    plt.xlabel('Time of Entry')
    plt.legend()
    plt.title('Total no. of infections vs Time of Entry (a0 = 0.0007)')
    plt.show()
    return

def run1():
    for i in range(8):
        main3(0.08,0.07,0.0003,200+i*100)

    plt.scatter(xvalues,area)
    plt.ylabel('Total no. of infections')
    plt.xlabel('Time of Entry')
    plt.legend()
    plt.title('Total no. of infections vs Time of Entry (a0 = 0.0003)')
    plt.show()
    return

run()
area.clear()
xvalues.clear()
run1()



In [ ]:
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt

def infectedfast(t,g,l,c,I_f):
    power = l/(g-l)
    value = (1 + c*math.exp((g-l)*t))/(1 + c)
    final = math.pow(value,power)*math.exp(-g*t)
    return I_f*final

def susceptiblefast(t,g,l,c,S_f):
    power = l/(g-l)
    value = (math.exp((l-g)*t) + c)/(1 + c)
    final = math.pow(value,power)
    return S_f*final

def recoveredfast(t,g,l,b,c,I_f):
    i = quad(infectedfast,0,t,args=(g,l,c,I_f))
    final = b*i[0]
    #power = g/(g-l)
    #value = (1 + c*math.exp((g-l)*t))/(1 + c)
    #final = ((1 + c)*b/g)*(math.exp(-g*t)*math.pow(value,power)-1)
    return final

def mintegrand(t,ls,g0,a0,g,l,c_f,I_f):
    c0 = a0*g0
    value = ls - (max(0.0001,g0 - c0*(infectedfast(t,g,l,c_f,I_f))))
    return value

def pintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    value = ls/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f))
    return value

def jintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    c0 = a0*g0
    value = ls*c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)) - max(0.0001,(g0 - c0*(infectedfast(s,g,l,c_f,I_f))))
    return value

def mvalue(parity,s,ls,g0,a0,g,l,c_f,I_f):
    i = quad(mintegrand,0,s,args=(ls,g0,a0,g,l,c_f,I_f))
    #print(temp)
    value = math.exp(parity*i[0])
    #print(i[0])
    return value

def pvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    k = -1
    i = quad(pintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(k*i[0])
    return value

def jvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    i = quad(jintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(i[0])
    return value

colors = ['b','g','r','c','m','y','k','w']

def main3(eta,zeta,a0):
    I_f = 1
    S_f = 9999
    c_f = S_f/I_f
    I_s = 1
    S_s = 9999
    c_s = S_s/I_s
    g = 1/15
    b = 1/20
    l = 0.25
    ls = eta*l
    g0 = zeta*g
  
    
   
    for i in range(6):
        enter = (i*200)+500
        #if i == 5:
            #enter = 601
        infected = [I_s]
        susceptible = [S_s]
        Ifast = [0]
        Sfast = [S_f]
        time = [0]
        for t in range(1700):
            if t <= enter:
                if t < enter:
                    Ifast.append(Ifast[t-1])
                    Sfast.append(Sfast[t-1])
                elif t == enter:
                    Ifast.append(I_f)
                    Sfast.append(S_f)
                infected.append(infectedfast(t+1,g0,ls,c_s,I_s))
                susceptible.append(susceptiblefast(t+1,g0,ls,c_s,S_s))
            if t > enter:
                Ifast.append(infectedfast(t-enter,g,l,c_f,I_f))
                Sfast.append(susceptiblefast(t-enter,g,l,c_f,S_f))
                I_s = infected[enter]
                S_s = susceptible[enter]
                c_s = S_s/I_s
                infected.append(I_s*jvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
                susceptible.append(S_s*pvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
            time.append(t+1)
        plt.plot(time,infected,label='t = '+str(enter),color = colors[i])
        #plt.plot(time,susceptible,label='Susceptible')
        #plt.plot(time,Ifast,label='FastInfected')
        #plt.plot(time,Sfast,label='FastSusceptible')
        plt.legend()
        plt.title('[+}Infected Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0))
        #plt.savefig('Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0)+'.png')
        I_s = 1
        S_s = 9999
        c_s = S_s/I_s
    plt.show()
    
    return

main3(0.08,0.07,0.0001)
main3(0.08,0.07,0.0007)
#main3(0.08,0.07,0.001)

In [ ]:
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt

def infectedfast(t,g,l,c,I_f):
    power = l/(g-l)
    value = (1 + c*math.exp((g-l)*t))/(1 + c)
    final = math.pow(value,power)*math.exp(-g*t)
    return I_f*final

def susceptiblefast(t,g,l,c,S_f):
    power = l/(g-l)
    value = (math.exp((l-g)*t) + c)/(1 + c)
    final = math.pow(value,power)
    return S_f*final

def recoveredfast(t,g,l,b,c,I_f):
    i = quad(infectedfast,0,t,args=(g,l,c,I_f))
    final = b*i[0]
    #power = g/(g-l)
    #value = (1 + c*math.exp((g-l)*t))/(1 + c)
    #final = ((1 + c)*b/g)*(math.exp(-g*t)*math.pow(value,power)-1)
    return final

def mintegrand(t,ls,g0,a0,g,l,c_f,I_f):
    c0 = a0*g0
    value = ls - (max(0.0001,g0 - c0*(infectedfast(t,g,l,c_f,I_f))))
    return value

def pintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    value = ls/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f))
    return value

def jintegrand(s,ls,g0,a0,g,l,c_f,c_s,I_f):
    c0 = a0*g0
    value = ls*c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)/(1 + c_s*mvalue(-1,s,ls,g0,a0,g,l,c_f,I_f)) - max(0.0001,(g0 - c0*(infectedfast(s,g,l,c_f,I_f))))
    return value

def mvalue(parity,s,ls,g0,a0,g,l,c_f,I_f):
    i = quad(mintegrand,0,s,args=(ls,g0,a0,g,l,c_f,I_f))
    #print(temp)
    value = math.exp(parity*i[0])
    #print(i[0])
    return value

def pvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    k = -1
    i = quad(pintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(k*i[0])
    return value

def jvalue(t,ls,g0,a0,g,l,c_f,c_s,I_f):
    i = quad(jintegrand,0,t,args=(ls,g0,a0,g,l,c_f,c_s,I_f))
    value = math.exp(i[0])
    return value

colors = ['b','g','r','c','m','y','k','w']

def main3(eta,zeta,a0):
    I_f = 1
    S_f = 9999
    c_f = S_f/I_f
    I_s = 1
    S_s = 9999
    c_s = S_s/I_s
    g = 1/15
    b = 1/20
    l = 0.25
    ls = eta*l
    g0 = zeta*g
  
    peak = []
    peaktime = []
    entrytime = []
   
    for i in range(8):
        enter = (i*50)+400
        #if i == 5:
            #enter = 601
        infected = [I_s]
        susceptible = [S_s]
        Ifast = [0]
        Sfast = [S_f]
        time = [0]
        for t in range(1700):
            if t <= enter:
                if t < enter:
                    Ifast.append(Ifast[t-1])
                    Sfast.append(Sfast[t-1])
                elif t == enter:
                    Ifast.append(I_f)
                    Sfast.append(S_f)
                infected.append(infectedfast(t+1,g0,ls,c_s,I_s))
                susceptible.append(susceptiblefast(t+1,g0,ls,c_s,S_s))
            if t > enter:
                Ifast.append(infectedfast(t-enter,g,l,c_f,I_f))
                Sfast.append(susceptiblefast(t-enter,g,l,c_f,S_f))
                I_s = infected[enter]
                S_s = susceptible[enter]
                c_s = S_s/I_s
                infected.append(I_s*jvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
                susceptible.append(S_s*pvalue(t-enter,ls,g0,a0,g,l,c_f,c_s,I_f))
            time.append(t+1)
        #plt.plot(time,infected,label='t = '+str(enter),color = colors[i])
        #plt.plot(time,susceptible,label='Susceptible')
        #plt.plot(time,Ifast,label='FastInfected')
        #plt.plot(time,Sfast,label='FastSusceptible')
        #plt.legend()
        #plt.title('[+}Infected Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0))
        #plt.savefig('Time = '+str(enter)+' Eta = '+str(eta)+' Zeta = '+str(zeta)+' a0 = '+str(a0)+'.png')
        I_s = 1
        S_s = 9999
        c_s = S_s/I_s
        peak.append(max(infected))
        peaktime.append(time[infected.index(max(infected))])
        entrytime.append(enter)
    fig1 = plt.figure('Figure1')
    plt.plot(entrytime,peak)
    plt.title('Peak vs Time of Entry')
    fig2 = plt.figure('Figure2')
    plt.plot(entrytime,peaktime)
    plt.title('Time of Peak vs Time of Entry')
    plt.show()
    return

#main3(0.08,0.07,0.0001)
#main3(0.08,0.07,0.0007)
main3(0.08,0.07,0.001)